In [27]:
import simfin as sf
import pandas as pd 
import numpy as np 
import hvplot.pandas
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import GradientBoostingClassifier
from collections import Counter

In [28]:
# Set API Key 
sf.set_api_key('ZLj424KDiohtSc0u9bVaF7mIeIemoAir')

In [29]:
# Make Directory for Data 
sf.set_data_dir('~/simfin_data/')

In [30]:
# Load Ratios Data 
ratios_df = sf.load_derived(variant='quarterly-asreported', market='us')
print(ratios_df.head())

Dataset "us-derived-quarterly-asreported" on disk (4 days old).
- Loading from disk ... Done!
                    SimFinId Currency  Fiscal Year Fiscal Period Publish Date  \
Ticker Report Date                                                              
A      2010-07-31      45846      USD         2010            Q3   2010-10-06   
       2010-10-31      45846      USD         2010            Q4   2010-12-20   
       2011-01-31      45846      USD         2011            Q1   2011-03-09   
       2011-04-30      45846      USD         2011            Q2   2011-06-07   
       2011-07-31      45846      USD         2011            Q3   2011-09-07   

                   Restated Date       EBITDA  Total Debt  Free Cash Flow  \
Ticker Report Date                                                          
A      2010-07-31     2011-09-07  175000000.0  3678000000     174000000.0   
       2010-10-31     2012-12-20  270000000.0  3691000000     258000000.0   
       2011-01-31     2012-03-

In [31]:
# Put Ratios Data in DataFrame
pd.DataFrame(ratios_df)

SimFinId Currency  Fiscal Year Fiscal Period Publish Date  \
Ticker Report Date                                                              
A      2010-07-31      45846      USD         2010            Q3   2010-10-06   
       2010-10-31      45846      USD         2010            Q4   2010-12-20   
       2011-01-31      45846      USD         2011            Q1   2011-03-09   
       2011-04-30      45846      USD         2011            Q2   2011-06-07   
       2011-07-31      45846      USD         2011            Q3   2011-09-07   
...                      ...      ...          ...           ...          ...   
ZYXI   2020-09-30     171401      USD         2020            Q3   2020-10-27   
       2020-12-31     171401      USD         2020            Q4   2021-02-25   
       2021-03-31     171401      USD         2021            Q1   2021-04-29   
       2021-06-30     171401      USD         2021            Q2   2021-07-29   
       2021-09-30     171401      USD         2021            Q3   2021-11-02   

                   Restated Date       EBITDA  Total Debt  Free Cash Flow  \
Ticker Report Date                                                          
A      2010-07-31     2011-09-07  175000000.0  3678000000     174000000.0   
       2010-10-31     2012-12-20  270000000.0  3691000000     258000000.0   
       2011-01-31     2012-03-05  274000000.0  2139000000      65000000.0   
       2011-04-30     2012-06-04  330000000.0  2144000000     393000000.0   
       2011-07-31     2012-09-05  344000000.0  2168000000      74000000.0   
...                          ...          ...         ...             ...   
ZYXI   2020-09-30     2020-10-27    1914000.0           0             NaN   
       2020-12-31     2021-02-25    2827000.0           0      -3070000.0   
       2021-03-31     2021-04-29    -594000.0           0      -6150000.0   
       2021-06-30     2021-07-29    4422000.0           0       1714000.0   
       2021-09-30     2021-11-02    8641000.0           0       6443000.0   

                    Gross Profit Margin  ...  Sales Per Share  \
Ticker Report Date                       ...                    
A      2010-07-31               0.52384  ...          3.98847   
       2010-10-31               0.52919  ...          4.58140   
       2011-01-31               0.53720  ...          4.37752   
       2011-04-30               0.53667  ...          4.83285   
       2011-07-31               0.52750  ...          4.85920   
...                                 ...  ...              ...   
ZYXI   2020-09-30               0.78548  ...          0.52791   
       2020-12-31               0.77899  ...          0.66900   
       2021-03-31               0.75604  ...          0.62961   
       2021-06-30               0.76575  ...          0.81016   
       2021-09-30               0.80346  ...          0.90956   

                    Equity Per Share  Free Cash Flow Per Share  \
Ticker Report Date                                               
A      2010-07-31            8.07493                   0.50144   
       2010-10-31            9.38372                   0.75000   
       2011-01-31            9.60231                   0.18732   
       2011-04-30           11.39193                   1.13256   
       2011-07-31           12.04598                   0.21264   
...                              ...                       ...   
ZYXI   2020-09-30            1.43170                       NaN   
       2020-12-31            1.48550                  -0.08021   
       2021-03-31            1.46534                  -0.16049   
       2021-06-30            1.49644                   0.04476   
       2021-09-30            1.65730                   0.16847   

                    Dividends Per Share  Piotroski F-Score  \
Ticker Report Date                                           
A      2010-07-31                   NaN                NaN   
       2010-10-31                   0.0                6.0   
       2011

In [32]:
# Reset ratios_df index 
ratios_df.reset_index(inplace=True)

# Filter for companies currently trading 
companies_dirty = ratios_df['Ticker'].to_list()

companies_clean = []
[companies_clean.append(x) for x in companies_dirty if x not in companies_clean]

char_list = ['_','1','ACPW', 'ADFK', 'ALTMS', 'ARTEC', 'AUDH', 'BCLR', 'BCSI', 'BLUD', 'C649', 'CCFI', 'CICC', 'CMBG', 'CYNX', 'DDMG', 'DGTC', 'DMDW', 'ELLI', 'EXDX', 'HOMR', 'INFOR', 'JCG', 'JDAS', 'LOXO', 'LTCH', 'MFE', 'NCSO', 'NDMT', 
             'NKT.CO', 'NOIZ', 'QVC', 'SCRH', 'SNGC', 'TM-28', 'TMB', 'VNTV', 'WLT', 'XING']
companies_clean = [ele for ele in companies_clean if all(ch not in ele for ch in char_list)]

# Rename date column and reindex, also drop bad columns 
ratios_df.rename(columns={'Publish Date':'Date'},inplace=True)
ratios_df.set_index(['Ticker', 'Date'], inplace=True)

ratios_df.drop(columns=['Free Cash Flow', 'EBITDA', 'Total Debt', 'SimFinId','Currency', 'Report Date','Restated Date'],inplace=True)
ratios_df.head()

Fiscal Year Fiscal Period  Gross Profit Margin  \
Ticker Date                                                         
A      2010-10-06         2010            Q3              0.52384   
       2010-12-20         2010            Q4              0.52919   
       2011-03-09         2011            Q1              0.53720   
       2011-06-07         2011            Q2              0.53667   
       2011-09-07         2011            Q3              0.52750   

                   Operating Margin  Net Profit Margin  Return on Equity  \
Ticker Date                                                                
A      2010-10-06           0.08309            0.14812           0.07295   
       2010-12-20           0.12881            0.18528           0.09023   
       2011-03-09           0.13891            0.12706           0.05780   
       2011-06-07           0.15862            0.11926           0.05049   
       2011-09-07           0.16617            0.19515           0.07857   

                   Return on Assets  Free Cash Flow to Net Income  \
Ticker Date                                                         
A      2010-10-06           0.02253                       0.84878   
       2010-12-20           0.03012                       0.88356   
       2011-03-09           0.02399                       0.33679   
       2011-06-07           0.02312                       1.96500   
       2011-09-07           0.03770                       0.22424   

                   Current Ratio  Liabilities to Equity Ratio  ...  \
Ticker Date                                                    ...   
A      2010-10-06        1.96606                      2.23843  ...   
       2010-12-20        2.00097                      1.99629  ...   
       2011-03-09        3.27027                      1.40910  ...   
       2011-06-07        3.20101                      1.18354  ...   
       2011-09-07        3.47043                      1.08405  ...   

                   Sales Per Share  Equity Per Share  \
Ticker Date                                            
A      2010-10-06          3.98847           8.07493   
       2010-12-20          4.58140           9.38372   
       2011-03-09          4.37752           9.60231   
       2011-06-07          4.83285          11.39193   
       2011-09-07          4.85920          12.04598   

                   Free Cash Flow Per Share  Dividends Per Share  \
Ticker Date                                                        
A      2010-10-06                   0.50144                  NaN   
       2010-12-20                   0.75000                  0.0   
       2011-03-09                   0.18732                  NaN   
       2011-06-07                   1.13256                  0.0   
       2011-09-07                   0.21264                  0.0   

                   Piotroski F-Score  Return On Invested Capital  \
Ticker Date                                                        
A      2010-10-06                NaN                     0.02668   
       2010-12-20                6.0                     0.03599   
       2011-03-09                NaN                     0.02907   
       2011-06-07                NaN                     0.02834   
       2011-09-07                6.0                     0.04553   

                   Cash Return On Invested Capital  Dividend Payout Ratio  \
Ticker Date                                                                 
A      2010-10-06                          0.02264                    NaN   
       2010-12-20                          0.03180                    0.0   
       2011-03-09                          0.00979                    NaN   
       2011-06-07                          0.05569                    0.0   
       2011-09-07                          0.01021                    0.0   

                   Net Debt / EBITDA  Net Debt / EBIT  
Ticker Date                                            
A      2010-10-06           

In [33]:
# Load prices data 
prices_df = sf.load_shareprices(variant='daily', market='us')


Dataset "us-shareprices-daily" on disk (4 days old).
- Loading from disk ... Done!


In [34]:
# Drop all columns except 

prices_df.columns
prices_df.drop(columns=['Open', 'Low', 'High', 'Close', 'Dividend',
       'Volume', 'Shares Outstanding','SimFinId'], inplace=True)

In [35]:
# Create Daily Return Column 
prices_df['pct_change']=prices_df['Adj. Close'].pct_change().shift(-1)



In [36]:
# Visualize prices_df
prices_df.head()



Adj. Close  pct_change
Ticker Date                              
A      2007-01-03       22.60    0.003097
       2007-01-04       22.67   -0.009263
       2007-01-05       22.46   -0.003562
       2007-01-08       22.38    0.001340
       2007-01-09       22.41   -0.008925

In [37]:
# Visualize ratios_df
ratios_df.head()


Fiscal Year Fiscal Period  Gross Profit Margin  \
Ticker Date                                                         
A      2010-10-06         2010            Q3              0.52384   
       2010-12-20         2010            Q4              0.52919   
       2011-03-09         2011            Q1              0.53720   
       2011-06-07         2011            Q2              0.53667   
       2011-09-07         2011            Q3              0.52750   

                   Operating Margin  Net Profit Margin  Return on Equity  \
Ticker Date                                                                
A      2010-10-06           0.08309            0.14812           0.07295   
       2010-12-20           0.12881            0.18528           0.09023   
       2011-03-09           0.13891            0.12706           0.05780   
       2011-06-07           0.15862            0.11926           0.05049   
       2011-09-07           0.16617            0.19515           0.07857   

                   Return on Assets  Free Cash Flow to Net Income  \
Ticker Date                                                         
A      2010-10-06           0.02253                       0.84878   
       2010-12-20           0.03012                       0.88356   
       2011-03-09           0.02399                       0.33679   
       2011-06-07           0.02312                       1.96500   
       2011-09-07           0.03770                       0.22424   

                   Current Ratio  Liabilities to Equity Ratio  ...  \
Ticker Date                                                    ...   
A      2010-10-06        1.96606                      2.23843  ...   
       2010-12-20        2.00097                      1.99629  ...   
       2011-03-09        3.27027                      1.40910  ...   
       2011-06-07        3.20101                      1.18354  ...   
       2011-09-07        3.47043                      1.08405  ...   

                   Sales Per Share  Equity Per Share  \
Ticker Date                                            
A      2010-10-06          3.98847           8.07493   
       2010-12-20          4.58140           9.38372   
       2011-03-09          4.37752           9.60231   
       2011-06-07          4.83285          11.39193   
       2011-09-07          4.85920          12.04598   

                   Free Cash Flow Per Share  Dividends Per Share  \
Ticker Date                                                        
A      2010-10-06                   0.50144                  NaN   
       2010-12-20                   0.75000                  0.0   
       2011-03-09                   0.18732                  NaN   
       2011-06-07                   1.13256                  0.0   
       2011-09-07                   0.21264                  0.0   

                   Piotroski F-Score  Return On Invested Capital  \
Ticker Date                                                        
A      2010-10-06                NaN                     0.02668   
       2010-12-20                6.0                     0.03599   
       2011-03-09                NaN                     0.02907   
       2011-06-07                NaN                     0.02834   
       2011-09-07                6.0                     0.04553   

                   Cash Return On Invested Capital  Dividend Payout Ratio  \
Ticker Date                                                                 
A      2010-10-06                          0.02264                    NaN   
       2010-12-20                          0.03180                    0.0   
       2011-03-09                          0.00979                    NaN   
       2011-06-07                          0.05569                    0.0   
       2011-09-07                          0.01021                    0.0   

                   Net Debt / EBITDA  Net Debt / EBIT  
Ticker Date                                            
A      2010-10-06           

In [38]:
# Combiner function can combine ratio_df with prices_df 

def combiner(ticker):
    ticker_ratio = ratios_df.loc[ticker,:]
    ticker_prices = prices_df.loc[ticker,:]
    tickers = pd.merge(ticker_ratio, ticker_prices,left_index=True, right_index=True,how='inner')
   
    return tickers
    

    

In [39]:
# Visualize list of companies currently trading 
companies_clean


['A',
 'AA',
 'AAL',
 'AAMC',
 'AAN',
 'AAOI',
 'AAON',
 'AAP',
 'AAPL',
 'AAWW',
 'ABAX',
 'ABBV',
 'ABC',
 'ABCD',
 'ABCL',
 'ABEO',
 'ABG',
 'ABIO',
 'ABM',
 'ABMC',
 'ABMD',
 'ABNB',
 'ABSI',
 'ABT',
 'ABTL',
 'ABUS',
 'AC',
 'ACA',
 'ACAD',
 'ACAN',
 'ACAT',
 'ACCO',
 'ACER',
 'ACET',
 'ACHC',
 'ACI',
 'ACIA',
 'ACIW',
 'ACLS',
 'ACM',
 'ACN',
 'ACRX',
 'ACT',
 'ACU',
 'ACW',
 'ACY',
 'ADBE',
 'ADES',
 'ADI',
 'ADIL',
 'ADM',
 'ADMA',
 'ADNT',
 'ADOM',
 'ADP',
 'ADPT',
 'ADS',
 'ADSK',
 'ADT',
 'ADTN',
 'ADUS',
 'ADV',
 'ADVM',
 'ADXS',
 'AE',
 'AEE',
 'AEHR',
 'AEIS',
 'AEO',
 'AEP',
 'AEPI',
 'AES',
 'AFI',
 'AFRM',
 'AGCO',
 'AGI',
 'AGIO',
 'AGLE',
 'AGN',
 'AGNC',
 'AGR',
 'AGS',
 'AGX',
 'AGYS',
 'AHPI',
 'AIMC',
 'AIR',
 'AIRI',
 'AIT',
 'AJG',
 'AJRD',
 'AKAM',
 'AKR',
 'AKRX',
 'AKS',
 'AKYA',
 'AL',
 'ALB',
 'ALCO',
 'ALE',
 'ALG',
 'ALGN',
 'ALGT',
 'ALJ',
 'ALJJ',
 'ALK',
 'ALKS',
 'ALLE',
 'ALNY',
 'ALOT',
 'ALPN',
 'ALR',
 'ALRM',
 'ALRN',
 'ALSK',
 'ALSN',
 'ALT',
 

In [55]:
# Pass the 'cleaned' list of company tickers through combiner 
all_df = combiner(companies_clean)

all_df.head()

Fiscal Year Fiscal Period  Gross Profit Margin  \
Ticker Date                                                         
A      2010-10-06         2010            Q3              0.52384   
       2010-12-20         2010            Q4              0.52919   
       2011-03-09         2011            Q1              0.53720   
       2011-06-07         2011            Q2              0.53667   
       2011-09-07         2011            Q3              0.52750   

                   Operating Margin  Net Profit Margin  Return on Equity  \
Ticker Date                                                                
A      2010-10-06           0.08309            0.14812           0.07295   
       2010-12-20           0.12881            0.18528           0.09023   
       2011-03-09           0.13891            0.12706           0.05780   
       2011-06-07           0.15862            0.11926           0.05049   
       2011-09-07           0.16617            0.19515           0.07857   

                   Return on Assets  Free Cash Flow to Net Income  \
Ticker Date                                                         
A      2010-10-06           0.02253                       0.84878   
       2010-12-20           0.03012                       0.88356   
       2011-03-09           0.02399                       0.33679   
       2011-06-07           0.02312                       1.96500   
       2011-09-07           0.03770                       0.22424   

                   Current Ratio  Liabilities to Equity Ratio  ...  \
Ticker Date                                                    ...   
A      2010-10-06        1.96606                      2.23843  ...   
       2010-12-20        2.00097                      1.99629  ...   
       2011-03-09        3.27027                      1.40910  ...   
       2011-06-07        3.20101                      1.18354  ...   
       2011-09-07        3.47043                      1.08405  ...   

                   Free Cash Flow Per Share  Dividends Per Share  \
Ticker Date                                                        
A      2010-10-06                   0.50144                  NaN   
       2010-12-20                   0.75000                  0.0   
       2011-03-09                   0.18732                  NaN   
       2011-06-07                   1.13256                  0.0   
       2011-09-07                   0.21264                  0.0   

                   Piotroski F-Score  Return On Invested Capital  \
Ticker Date                                                        
A      2010-10-06                NaN                     0.02668   
       2010-12-20                6.0                     0.03599   
       2011-03-09                NaN                     0.02907   
       2011-06-07                NaN                     0.02834   
       2011-09-07                6.0                     0.04553   

                   Cash Return On Invested Capital  Dividend Payout Ratio  \
Ticker Date                                                                 
A      2010-10-06                          0.02264                    NaN   
       2010-12-20                          0.03180                    0.0   
       2011-03-09                          0.00979                    NaN   
       2011-06-07                          0.05569                    0.0   
       2011-09-07                          0.01021                    0.0   

                   Net Debt / EBITDA  Net Debt / EBIT  Adj. Close  pct_change  
Ticker Date                                                                    
A      2010-10-06            7.77714         11.83478       21.53    0.016256  
       2010-12-20            3.85926          5.13300       26.77    0.004856  
       2011-03-09           -1.82117         -2.36493       30.34   -0.025379  
       2011-06-07           -2.51818         -3.12406       31.43    0.000955  
       2011-09-07           -2.71221         -3.32028

In [72]:
#all_df_viz = all_df.drop(columns=['Fiscal Year', 'Fiscal Period', 'Dividends Per Share', 'Piotroski F-Score', 'Dividend Payout Ratio'],inplace=True)

all_df_viz = all_df

#all_df_viz.drop(columns=['Adj. Close'],inplace=True)
means = all_df_viz.mean()

means.hvplot.bar(ylabel='$',rot=90)


:Bars   [index]   (0)

In [18]:



#all_df.drop(columns=['Fiscal Year','Fiscal Period','SimFinId','Adj. Close','Dividends Per Share', 'Dividend Payout Ratio'],inplace=True)

# Make positive return into '1' and negative return into '0' 

all_df['pct_change'][all_df['pct_change'] < 0] = 0
all_df['pct_change'][all_df['pct_change'] > 0] = 1
all_df.head()

/var/folders/ql/7cpmkpc96jq83bztknmmyg880000gn/T/ipykernel_56727/531027856.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_df['pct_change'][all_df['pct_change'] < 0] = 0
/var/folders/ql/7cpmkpc96jq83bztknmmyg880000gn/T/ipykernel_56727/531027856.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_df['pct_change'][all_df['pct_change'] > 0] = 1


Fiscal Year Fiscal Period  Gross Profit Margin  \
Ticker Date                                                         
A      2010-10-06         2010            Q3              0.52384   
       2010-12-20         2010            Q4              0.52919   
       2011-03-09         2011            Q1              0.53720   
       2011-06-07         2011            Q2              0.53667   
       2011-09-07         2011            Q3              0.52750   

                   Operating Margin  Net Profit Margin  Return on Equity  \
Ticker Date                                                                
A      2010-10-06           0.08309            0.14812           0.07295   
       2010-12-20           0.12881            0.18528           0.09023   
       2011-03-09           0.13891            0.12706           0.05780   
       2011-06-07           0.15862            0.11926           0.05049   
       2011-09-07           0.16617            0.19515           0.07857   

                   Return on Assets  Free Cash Flow to Net Income  \
Ticker Date                                                         
A      2010-10-06           0.02253                       0.84878   
       2010-12-20           0.03012                       0.88356   
       2011-03-09           0.02399                       0.33679   
       2011-06-07           0.02312                       1.96500   
       2011-09-07           0.03770                       0.22424   

                   Current Ratio  Liabilities to Equity Ratio  ...  \
Ticker Date                                                    ...   
A      2010-10-06        1.96606                      2.23843  ...   
       2010-12-20        2.00097                      1.99629  ...   
       2011-03-09        3.27027                      1.40910  ...   
       2011-06-07        3.20101                      1.18354  ...   
       2011-09-07        3.47043                      1.08405  ...   

                   Free Cash Flow Per Share  Dividends Per Share  \
Ticker Date                                                        
A      2010-10-06                   0.50144                  NaN   
       2010-12-20                   0.75000                  0.0   
       2011-03-09                   0.18732                  NaN   
       2011-06-07                   1.13256                  0.0   
       2011-09-07                   0.21264                  0.0   

                   Piotroski F-Score  Return On Invested Capital  \
Ticker Date                                                        
A      2010-10-06                NaN                     0.02668   
       2010-12-20                6.0                     0.03599   
       2011-03-09                NaN                     0.02907   
       2011-06-07                NaN                     0.02834   
       2011-09-07                6.0                     0.04553   

                   Cash Return On Invested Capital  Dividend Payout Ratio  \
Ticker Date                                                                 
A      2010-10-06                          0.02264                    NaN   
       2010-12-20                          0.03180                    0.0   
       2011-03-09                          0.00979                    NaN   
       2011-06-07                          0.05569                    0.0   
       2011-09-07                          0.01021                    0.0   

                   Net Debt / EBITDA  Net Debt / EBIT  Adj. Close  pct_change  
Ticker Date                                                                    
A      2010-10-06            7.77714         11.83478       21.53         1.0  
       2010-12-20            3.85926          5.13300       26.77         1.0  
       2011-03-09           -1.82117         -2.36493       30.34         0.0  
       2011-06-07           -2.51818         -3.12406       31.43         1.0  
       2011-09-07           -2.71221         -3.32028

In [19]:
all_df.reset_index(drop=True, inplace=True)
all_df.drop(columns=['Fiscal Year', 'Fiscal Period', 'Adj. Close', 'Dividends Per Share', 'Piotroski F-Score','Dividend Payout Ratio'],inplace=True)

all_df.head()

,Gross Profit Margin,Operating Margin,Net Profit Margin,Return on Equity,Return on Assets,Free Cash Flow to Net Income,Current Ratio,Liabilities to Equity Ratio,Debt Ratio,"Earnings Per Share, Basic","Earnings Per Share, Diluted",Sales Per Share,Equity Per Share,Free Cash Flow Per Share,Return On Invested Capital,Cash Return On Invested Capital,Net Debt / EBITDA,Net Debt / EBIT,pct_change
0,0.52384,0.08309,0.14812,0.07295,0.02253,0.84878,1.96606,2.23843,0.40418,0.59078,0.58239,3.98847,8.07493,0.50144,0.02668,0.02264,7.77714,11.83478,1.0
1,0.52919,0.12881,0.18528,0.09023,0.03012,0.88356,2.00097,1.99629,0.38067,0.84884,0.82022,4.58140,9.38372,0.75000,0.03599,0.03180,3.85926,5.13300,1.0
2,0.53720,0.13891,0.12706,0.05780,0.02399,0.33679,3.27027,1.40910,0.26591,0.55620,0.54366,4.37752,9.60231,0.18732,0.02907,0.00979,-1.82117,-2.36493,0.0
3,0.53667,0.15862,0.11926,0.05049,0.02312,1.96500,3.20101,1.18354,0.24789,0.57637,0.56338,4.83285,11.39193,1.13256,0.02834,0.05569,-2.51818,-3.12406,1.0
4,0.52750,0.16617,0.19515,0.07857,0.03770,0.22424,3.47043,1.08405,0.24769,0.94828,0.92437,4.85920,12.04598,0.21264,0.04553,0.01021,-2.71221,-3.32028,0.0


In [20]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [21]:
all_df = clean_dataset(all_df)
all_df.head()

,Gross Profit Margin,Operating Margin,Net Profit Margin,Return on Equity,Return on Assets,Free Cash Flow to Net Income,Current Ratio,Liabilities to Equity Ratio,Debt Ratio,"Earnings Per Share, Basic","Earnings Per Share, Diluted",Sales Per Share,Equity Per Share,Free Cash Flow Per Share,Return On Invested Capital,Cash Return On Invested Capital,Net Debt / EBITDA,Net Debt / EBIT,pct_change
0,0.52384,0.08309,0.14812,0.07295,0.02253,0.84878,1.96606,2.23843,0.40418,0.59078,0.58239,3.98847,8.07493,0.50144,0.02668,0.02264,7.77714,11.83478,1.0
1,0.52919,0.12881,0.18528,0.09023,0.03012,0.88356,2.00097,1.99629,0.38067,0.84884,0.82022,4.58140,9.38372,0.75000,0.03599,0.03180,3.85926,5.13300,1.0
2,0.53720,0.13891,0.12706,0.05780,0.02399,0.33679,3.27027,1.40910,0.26591,0.55620,0.54366,4.37752,9.60231,0.18732,0.02907,0.00979,-1.82117,-2.36493,0.0
3,0.53667,0.15862,0.11926,0.05049,0.02312,1.96500,3.20101,1.18354,0.24789,0.57637,0.56338,4.83285,11.39193,1.13256,0.02834,0.05569,-2.51818,-3.12406,1.0
4,0.52750,0.16617,0.19515,0.07857,0.03770,0.22424,3.47043,1.08405,0.24769,0.94828,0.92437,4.85920,12.04598,0.21264,0.04553,0.01021,-2.71221,-3.32028,0.0


Gross Profit Margin                  0.430979
Operating Margin                    -0.121959
Net Profit Margin                    0.022417
Return on Equity                    -0.284654
Return on Assets                     0.452606
Free Cash Flow to Net Income         1.933255
Current Ratio                        2.500911
Liabilities to Equity Ratio          4.081912
Debt Ratio                           0.255957
Earnings Per Share, Basic           -5.624222
Earnings Per Share, Diluted         -5.663517
Sales Per Share                     39.086758
Equity Per Share                   103.698407
Free Cash Flow Per Share            -2.242419
Return On Invested Capital          -0.748100
Cash Return On Invested Capital     -0.740558
Net Debt / EBITDA                   14.820871
Net Debt / EBIT                     57.671102
pct_change                           0.495056
dtype: float64